In [2]:
import tensorflow as tf
import pandas as pd
from datasets import load_dataset

In [3]:
dataset  = load_dataset('banking77')

Using custom data configuration default
Reusing dataset banking77 (C:\Users\barka\.cache\huggingface\datasets\banking77\default\1.1.0\aec0289529599d4572d76ab00c8944cb84f88410ad0c9e7da26189d31f62a55b)


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
train_df = pd.DataFrame(dataset['train'])
test_df = pd.DataFrame( dataset['test'])

In [138]:
from sklearn.model_selection import train_test_split

X_train,X_valid,y_train,y_valid = train_test_split(train_df['text'],train_df['label'],stratify=train_df['label'],test_size=0.1,random_state=42)

In [139]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = tf.data.Dataset.from_tensor_slices((X_train,y_train)).batch(32).cache().prefetch(buffer_size=AUTOTUNE)
val_ds = tf.data.Dataset.from_tensor_slices((X_valid,y_valid)).batch(32).cache().prefetch(buffer_size=AUTOTUNE)
test_ds = tf.data.Dataset.from_tensor_slices((test_df['text'],test_df['label'])).batch(32).cache().prefetch(buffer_size=AUTOTUNE,)

In [128]:
text_batch_1, text_label_1 = next(iter(train_ds)) # pick first batch from dataset
text , label = text_batch_1[0],text_label_1[0] # taking first text and label from batch
text , label

(<tf.Tensor: shape=(), dtype=string, numpy=b'I want to auto top-up.'>,
 <tf.Tensor: shape=(), dtype=int64, numpy=4>)

In [27]:
import tensorflow_text as text  # Registers the ops.

In [140]:
import tensorflow_hub as hub
text_input = tf.keras.Input(shape=(),dtype=tf.string,name="input_layer")
preprocessor = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',name="preprocessor")
encoder_inputs = preprocessor(text_input)

encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2',trainable=True,name='BERT_encoder')
outputs = encoder(encoder_inputs)

pooled_output = outputs['pooled_output']

#create Dense layer for classifier
# net = tf.keras.layers.Dropout(0.1)(pooled_output)
net = tf.keras.layers.Dense(77,activation='softmax')(pooled_output)




In [176]:
train_df.loc[2,'text']

'I have been waiting over a week. Is the card still coming?'

In [177]:
embedding_model = tf.keras.Model(text_input, net)
sentences = tf.constant(["I have been waiting over a week. Is the card still coming?"])
print(embedding_model(sentences) , tf.math.argmax(embedding_model(sentences),1) )

tf.Tensor(
[[0.01573822 0.00680786 0.00682698 0.0057435  0.00663491 0.00416449
  0.00671658 0.00300336 0.00776005 0.02080373 0.00809656 0.01633636
  0.01218717 0.02530308 0.00674909 0.10578006 0.01120544 0.0274372
  0.02170157 0.01573708 0.01062904 0.00761745 0.00695283 0.00173848
  0.01317095 0.00980883 0.01405094 0.00347213 0.01154466 0.00719797
  0.00762379 0.01444083 0.00922981 0.02476273 0.002159   0.00631858
  0.00529782 0.01209983 0.01630371 0.03186492 0.02393494 0.04164134
  0.0033001  0.02736972 0.00330217 0.01364071 0.0066231  0.00550543
  0.00821105 0.00769989 0.01056043 0.00515484 0.01237898 0.01605432
  0.02369267 0.00198569 0.01726761 0.01083891 0.01656568 0.01373474
  0.01385983 0.00742864 0.00661014 0.01571984 0.01459891 0.01259194
  0.00986702 0.01351462 0.00586727 0.00526361 0.00591046 0.01368266
  0.005771   0.01596901 0.00332802 0.0143096  0.01522762]], shape=(1, 77), dtype=float32) tf.Tensor([15], shape=(1,), dtype=int64)


In [38]:
embedding_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_layer (InputLayer)       [(None,)]            0           []                               
                                                                                                  
 preprocessor (KerasLayer)      {'input_type_ids':   0           ['input_layer[0][0]']            
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                    

In [43]:
TOTAL_SIZE = train_df.shape[0]
BATCH_SIZE = 32
VAL_BATCH  = int((TOTAL_SIZE / BATCH_SIZE) * 0.9) 

In [142]:
int((TOTAL_SIZE / BATCH_SIZE) * 0.8)

250

In [58]:
train_ds = train_ds.take(VAL_BATCH)
val_ds = train_ds.skip(VAL_BATCH)

In [ ]:
epochs = 5
# steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
# num_train_steps = steps_per_epoch * epochs
# num_warmup_steps = int(0.1*num_train_steps)

# init_lr = 3e-5
# optimizer = tf.keras.optimization.create_optimizer(init_lr=init_lr,
#                                           num_train_steps=num_train_steps,
#                                           num_warmup_steps=num_warmup_steps,
#                                           optimizer_type='adamw')

In [143]:
embedding_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                         loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                         metrics=['accuracy'])


In [144]:
embedding_model.fit(train_ds,validation_data=val_ds,epochs=5)

Epoch 1/5
282/282 [==============================] - 73s 252ms/step - loss: 4.5300 - accuracy: 0.0163 - val_loss: 4.3697 - val_accuracy: 0.0310
Epoch 2/5
282/282 [==============================] - 73s 257ms/step - loss: 4.3042 - accuracy: 0.0401 - val_loss: 4.1415 - val_accuracy: 0.0579
Epoch 3/5
282/282 [==============================] - 73s 260ms/step - loss: 4.1321 - accuracy: 0.0625 - val_loss: 3.9518 - val_accuracy: 0.1049
Epoch 4/5
282/282 [==============================] - 70s 250ms/step - loss: 3.9526 - accuracy: 0.1019 - val_loss: 3.7399 - val_accuracy: 0.1628
Epoch 5/5
282/282 [==============================] - 70s 248ms/step - loss: 3.7677 - accuracy: 0.1501 - val_loss: 3.5169 - val_accuracy: 0.2288


In [145]:
loss, accuracy = embedding_model.evaluate(test_ds)

97/97 [==============================] - 6s 67ms/step - loss: 3.5506 - accuracy: 0.2000


In [93]:
preds = embedding_model.predict(test_ds)

97/97 [==============================] - 6s 60ms/step


In [104]:
v = test_ds.take(1)

In [ ]:
for x,y in train_ds.take(1):
    print(x,y)
    # break

In [105]:
embedding_model.predict(v)

1/1 [==============================] - 0s 77ms/step


array([[0.06842163, 0.00334039, 0.00698865, ..., 0.00877284, 0.00494816,
        0.00503086],
       [0.03302078, 0.00379312, 0.0133736 , ..., 0.00748981, 0.02237591,
        0.01538504],
       [0.05113184, 0.00421031, 0.00963745, ..., 0.00564119, 0.02267505,
        0.00645326],
       ...,
       [0.04194529, 0.00361456, 0.00887997, ..., 0.01052712, 0.0054382 ,
        0.00625578],
       [0.02375717, 0.00542869, 0.01179141, ..., 0.00801594, 0.02415427,
        0.01230364],
       [0.01862687, 0.00712863, 0.01008906, ..., 0.00598353, 0.02096801,
        0.0128925 ]], dtype=float32)

In [94]:
preds = tf.math.argmax(preds,1)

In [110]:
preds[:32]

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([ 0, 53, 53, 36, 53,  0, 47,  0, 15, 47, 21, 52, 45, 45, 47, 45, 30,
       53, 47, 30, 30,  0, 53, 30, 63, 15, 30, 30, 30, 58, 53, 30],
      dtype=int64)>

In [87]:
for x,_ in test_ds.take(1):
    print(x,_)

tf.Tensor(
[b'How do I locate my card?'
 b'I still have not received my new card, I ordered over a week ago.'
 b'I ordered a card but it has not arrived. Help please!'
 b'Is there a way to know when my card will arrive?'
 b'My card has not arrived yet.' b'When will I get my card?'
 b'Do you know if there is a tracking number for the new card you sent me?'
 b'i have not received my card' b'still waiting on that card'
 b'Is it normal to have to wait over a week for my new card?'
 b'How do I track my card?' b'How long does a card delivery take?'
 b"I still don't have my card after 2 weeks.  What should I do?"
 b'still waiting on my new card'
 b'I am still waiting for my card after 1 week.  Is this ok?'
 b'I have been waiting longer than expected for my bank card, could you provide information on when it will arrive?'
 b"I've been waiting longer than expected for my card."
 b"Why hasn't my card been delivered?"
 b'Where is my new card? I have been waiting a week!'
 b"My card still hasn't a

In [147]:
preprocessor_model = hub.load(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
special_tokens_dict = preprocessor_model.tokenize.get_special_tokens_dict()

In [ ]:
text_inputs = [tf.keras.layers.Input(shape=(), dtype=tf.string)]
tokenize = hub.KerasLayer(preprocessor_model.tokenize)
tokenized_inputs = [tokenize(segment) for segment in text_inputs]

# Step 2 (optional): modify tokenized inputs.
pass

# Step 3: pack input sequences for the Transformer encoder.
seq_length = 128  # Your choice here.
bert_pack_inputs = hub.KerasLayer(
    preprocessor_model.bert_pack_inputs,
    arguments=dict(seq_length=seq_length))  # Optional argument.
encoder_inputs = bert_pack_inputs(tokenized_inputs)

In [ ]:
encoder_inputs(tf.constant(["I was expecting my new card and am wondering why I haven't received it yet?"]))

In [167]:
encoder_inputs

{'input_word_ids': <KerasTensor: shape=(None, 128) dtype=int32 (created by layer 'keras_layer_7')>,
 'input_type_ids': <KerasTensor: shape=(None, 128) dtype=int32 (created by layer 'keras_layer_7')>,
 'input_mask': <KerasTensor: shape=(None, 128) dtype=int32 (created by layer 'keras_layer_7')>}